In [10]:
class LoopBoundEntry:
    def __init__(self, pc):
        self.pc = pc  # 程序计数器 (Program Counter)
        self.iteration_counter = 0  # 迭代计数器，用于LRU判断
        self.loop_increment = 0  # 循环增量
        self.confidence = 0  # 置信度
        self.loop_param = 0  # 表示循环变量
        self.loop_bound = 0  # 表示循环边界

    def update_iteration(self, current_loop_param, current_loop_bound):
        """
        更新迭代信息，根据源值A和B更新增量与边界
        """
        if self.loop_increment == 0 and current_loop_param != self.loop_param:
            # 在第一次更新时计算循环增量
            self.loop_increment = abs(current_loop_param - self.loop_param)
        self.loop_param = current_loop_param
        self.loop_bound = current_loop_bound
        self.iteration_counter += 1  # 每次更新迭代，增加计数器

    def calculate_remaining_iterations(self):
        """
        计算剩余的迭代次数，假设 loop_bound 是循环边界
        """
        if self.loop_increment == 0:
            return "无法计算：增量为0"
        remaining_iterations = (self.loop_bound - self.loop_param) // self.loop_increment
        return max(remaining_iterations, 0)

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, "
                f"Iteration: {self.iteration_counter}, Increment: {self.loop_increment}, "
                f"Confidence: {self.confidence}, loop_param: {self.loop_param}, "
                f"loop_bound: {self.loop_bound}")


class LoopBoundDetector:
    def __init__(self, max_entries=16):
        self.entries = {}  # 存储所有的循环条目，按PC索引
        self.max_entries = max_entries  # 最大条目数

    def update_entry(self, pc, current_loop_param, current_loop_bound):
        """
        更新或创建一个新的LoopBoundEntry，基于程序计数器PC。
        """
        if pc not in self.entries:
            if len(self.entries) >= self.max_entries:
                # 如果超过最大条目数，执行LRU替换，根据iteration_counter选择最小的条目
                lru_pc = min(self.entries, key=lambda k: self.entries[k].iteration_counter)
                del self.entries[lru_pc]
                print(f"LRU: 删除最久未使用的条目，PC: {hex(lru_pc)}")
            # 创建新的条目
            self.entries[pc] = LoopBoundEntry(pc)

        # 更新条目的迭代信息
        entry = self.entries[pc]
        entry.update_iteration(current_loop_param, current_loop_bound)

    def get_remaining_iterations(self, pc):
        """
        获取指定PC的条目并计算剩余的迭代次数
        """
        if pc in self.entries:
            return self.entries[pc].calculate_remaining_iterations()
        return "PC不存在"
    
    def __repr__(self):
        """
        返回LoopBoundDetector的可读字符串表示，展示所有条目和访问顺序。
        """
        entries_str = "\n".join([f"{pc}: {repr(entry)}" for pc, entry in self.entries.items()])
        return (f"LoopBoundDetector(max_entries={self.max_entries}, "
                f"entries={{\n{entries_str}\n}})")


# 示例使用
if __name__ == "__main__": 
    detector = LoopBoundDetector(max_entries=3)

    # 测试用例 1: 基本循环更新
    print("测试用例 1: 基本循环更新")
    detector.update_entry(0x01, 10, 20)  # 初始循环变量10，边界20
    detector.update_entry(0x01, 12, 20)  # 循环变量增量为2
    remaining = detector.get_remaining_iterations(0x01)
    print(f"PC 0x01 剩余的迭代次数: {remaining}")  # 应输出 4
    print(detector)

    # 测试用例 2: 增量为0的特殊情况
    print("\n测试用例 2: 增量为0的特殊情况")
    detector.update_entry(0x02, 5, 5)  # 初始循环变量与边界相等
    remaining = detector.get_remaining_iterations(0x02)
    print(f"PC 0x02 剩余的迭代次数: {remaining}")  # 应输出 0，循环已完成
    print(detector)

    # 测试用例 3: 无法计算增量的情况
    print("\n测试用例 3: 无法计算增量的情况")
    detector.update_entry(0x03, 10, 30)  # 初始循环变量为10，边界为30
    remaining = detector.get_remaining_iterations(0x03)
    print(f"PC 0x03 剩余的迭代次数: {remaining}")  # 因增量为0，应输出 “无法计算：增量为0”
    detector.update_entry(0x03, 10, 30)  # 循环变量未变
    remaining = detector.get_remaining_iterations(0x03)
    print(f"PC 0x03 剩余的迭代次数: {remaining}")  # 应该仍然无法计算，因为增量未改变
    print(detector)

    # 测试用例 4: 正常计算增量和剩余迭代
    print("\n测试用例 4: 正常计算增量和剩余迭代")
    detector.update_entry(0x04, 10, 30)  # 初始循环变量为10，边界为30
    detector.update_entry(0x04, 15, 30)  # 增量为5
    remaining = detector.get_remaining_iterations(0x04)
    print(f"PC 0x04 剩余的迭代次数: {remaining}")  # 应输出 3
    print(detector)

    # 测试用例 5: 超过边界的情况
    print("\n测试用例 5: 超过边界的情况")
    detector.update_entry(0x05, 35, 30)  # 当前值超过边界
    remaining = detector.get_remaining_iterations(0x05)
    print(f"PC 0x05 剩余的迭代次数: {remaining}")  # 应输出 0，循环已经结束
    print(detector)

    # 测试用例 6: 测试基于iteration_counter的LRU替换策略
    print("\n测试用例 6: 测试LRU替换策略")
    detector.update_entry(0x06, 10, 50)  # PC 0x06
    detector.update_entry(0x07, 5, 25)  # PC 0x07
    detector.update_entry(0x08, 15, 40)  # PC 0x08
    print(f"当前条目数量: {len(detector.entries)}")  # 应为3
    print(detector)

    # 插入新条目，触发基于iteration_counter的LRU替换
    detector.update_entry(0x09, 12, 36)  # 插入新的PC 0x09，应该替换迭代次数最少的条目
    print(f"插入新条目后，当前条目数量: {len(detector.entries)}")  # 应为3
    remaining = detector.get_remaining_iterations(0x06)
    print(f"访问被替换的PC 0x06: {remaining}")  # 应该返回 "PC不存在"

    print(detector)


测试用例 1: 基本循环更新
PC 0x01 剩余的迭代次数: 0
LoopBoundDetector(max_entries=3, entries={
1: PC: 0x1, Iteration: 2, Increment: 10, Confidence: 0, loop_param: 12, loop_bound: 20
})

测试用例 2: 增量为0的特殊情况
PC 0x02 剩余的迭代次数: 0
LoopBoundDetector(max_entries=3, entries={
1: PC: 0x1, Iteration: 2, Increment: 10, Confidence: 0, loop_param: 12, loop_bound: 20
2: PC: 0x2, Iteration: 1, Increment: 5, Confidence: 0, loop_param: 5, loop_bound: 5
})

测试用例 3: 无法计算增量的情况
PC 0x03 剩余的迭代次数: 2
PC 0x03 剩余的迭代次数: 2
LoopBoundDetector(max_entries=3, entries={
1: PC: 0x1, Iteration: 2, Increment: 10, Confidence: 0, loop_param: 12, loop_bound: 20
2: PC: 0x2, Iteration: 1, Increment: 5, Confidence: 0, loop_param: 5, loop_bound: 5
3: PC: 0x3, Iteration: 2, Increment: 10, Confidence: 0, loop_param: 10, loop_bound: 30
})

测试用例 4: 正常计算增量和剩余迭代
LRU: 删除最久未使用的条目，PC: 0x2
PC 0x04 剩余的迭代次数: 1
LoopBoundDetector(max_entries=3, entries={
1: PC: 0x1, Iteration: 2, Increment: 10, Confidence: 0, loop_param: 12, loop_bound: 20
3: PC: 0x3, Iteration: 